## Recomendação de Produtos com KNN

### Bibliotecas utilizadas

In [6]:
import pandas as pd
import tkinter as tk
from sklearn.neighbors import NearestNeighbors
import numpy as np

### Base de dados

In [7]:
# Criando a nova base de dados fictícia com produtos eletrônicos e características adicionais
data = {
    'Produto': ['Smartphone', 'Notebook', 'Smartwatch', 'Tablet', 'Fones de Ouvido', 'Câmera', 'Carregador Portátil', 'Mouse', 'Teclado',
                'Monitor', 'Impressora', 'Roteador', 'Caixa de Som', 'Headset', 'Webcam'],
    'Categoria': ['Celulares', 'Computadores', 'Acessórios', 'Tablets', 'Acessórios', 'Câmeras', 'Acessórios', 'Acessórios', 'Acessórios',
                  'Monitores', 'Impressoras', 'Redes', 'Acessórios', 'Acessórios', 'Câmeras'],
    'Marca': ['Samsung', 'Dell', 'Apple', 'Samsung', 'Sony', 'Canon', 'Xiaomi', 'Logitech', 'Logitech',
              'LG', 'HP', 'TP-Link', 'JBL', 'Sony', 'Logitech'],
    'Preço': [1500, 3000, 400, 800, 100, 1200, 50, 20, 15, 500, 200, 80, 150, 50, 100],
    'Avaliação': [4.5, 4.8, 4.0, 4.2, 4.6, 4.7, 4.1, 4.3, 4.4, 4.5, 4.2, 4.6, 4.8, 4.3, 4.0]
}

df = pd.DataFrame(data)


### Normalizar dados numéricos

In [8]:
# Função para normalizar os dados numéricos
def normalizar(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

# Aplicar a normalização aos dados
df[['Preço', 'Avaliação']] = df[['Preço', 'Avaliação']].apply(normalizar)

### Função para recomendação

In [9]:
# Função para encontrar um produto mais similar, considerando múltiplas características
def encontrar_produto_similar(produto):
    # Obtendo a categoria do produto
    categoria_produto = df.loc[df['Produto'] == produto, 'Categoria'].values[0]
    
    # Filtrando a base de dados para conter todos os produtos
    produtos_categoria = df['Produto'].unique()
    
    # Obtendo os vetores de características dos produtos
    vetor_caracteristicas = pd.get_dummies(df[['Produto', 'Categoria']])
    
    # Adicionando os preços e avaliações de cada produto
    vetor_caracteristicas['Preço'] = df['Preço']
    vetor_caracteristicas['Avaliação'] = df['Avaliação']
    
    # Ponderando as características
    vetor_caracteristicas['Categoria_' + categoria_produto] *= 2  # Duplica o peso da categoria do produto
    
    vetor_caracteristicas = vetor_caracteristicas.values
    
    # Treinando o modelo KNN
    knn_model = NearestNeighbors(n_neighbors=3, algorithm='brute', metric='manhattan')
    knn_model.fit(vetor_caracteristicas)
    
    # Encontrando os vizinhos mais próximos
    produto_index = list(produtos_categoria).index(produto)
    _, indices = knn_model.kneighbors([vetor_caracteristicas[produto_index]])
    
    # Extraindo os produtos similares (excluindo o próprio produto)
    produtos_similares = [produtos_categoria[i] for i in indices.flatten()[1:]]
    
    return produtos_similares


### Interface Gráfica

In [10]:
# Criando a interface gráfica
def recomendar_produto():
    produto_comprado = entry_produto.get()
    produtos_recomendados = encontrar_produto_similar(produto_comprado)
    label_resultado.config(text=f"Produtos similares ao '{produto_comprado}': {', '.join(produtos_recomendados)}")

# Configurações da janela
root = tk.Tk()
root.title("Sistema de Recomendação de Produtos Eletrônicos")
root.geometry("400x200")

# Elementos da interface
label_produto = tk.Label(root, text="Produto Comprado:")
label_produto.pack(pady=5)

entry_produto = tk.Entry(root, width=30)
entry_produto.pack(pady=5)

button_recomendar = tk.Button(root, text="Recomendar", command=recomendar_produto)
button_recomendar.pack(pady=5)

label_resultado = tk.Label(root, text="")
label_resultado.pack(pady=5)

# Rodar a interface
root.mainloop()


### Código completo

In [ ]:
import pandas as pd
import tkinter as tk
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Criando a nova base de dados fictícia com produtos eletrônicos e características adicionais
data = {
    'Produto': ['Smartphone', 'Notebook', 'Smartwatch', 'Tablet', 'Fones de Ouvido', 'Câmera', 'Carregador Portátil', 'Mouse', 'Teclado',
                'Monitor', 'Impressora', 'Roteador', 'Caixa de Som', 'Headset', 'Webcam'],
    'Categoria': ['Celulares', 'Computadores', 'Acessórios', 'Tablets', 'Acessórios', 'Câmeras', 'Acessórios', 'Acessórios', 'Acessórios',
                  'Monitores', 'Impressoras', 'Redes', 'Acessórios', 'Acessórios', 'Câmeras'],
    'Marca': ['Samsung', 'Dell', 'Apple', 'Samsung', 'Sony', 'Canon', 'Xiaomi', 'Logitech', 'Logitech',
              'LG', 'HP', 'TP-Link', 'JBL', 'Sony', 'Logitech'],
    'Preço': [1500, 3000, 400, 800, 100, 1200, 50, 20, 15, 500, 200, 80, 150, 50, 100],
    'Avaliação': [4.5, 4.8, 4.0, 4.2, 4.6, 4.7, 4.1, 4.3, 4.4, 4.5, 4.2, 4.6, 4.8, 4.3, 4.0]
}

df = pd.DataFrame(data)

# Função para normalizar os dados numéricos
def normalizar(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

# Aplicar a normalização aos dados
df[['Preço', 'Avaliação']] = df[['Preço', 'Avaliação']].apply(normalizar)

# Função para encontrar um produto mais similar, considerando múltiplas características
def encontrar_produto_similar(produto):
    # Obtendo a categoria do produto
    categoria_produto = df.loc[df['Produto'] == produto, 'Categoria'].values[0]
    
    # Filtrando a base de dados para conter todos os produtos
    produtos_categoria = df['Produto'].unique()
    
    # Obtendo os vetores de características dos produtos
    vetor_caracteristicas = pd.get_dummies(df[['Produto', 'Categoria']])
    vetor_caracteristicas['Preço'] = df['Preço']
    vetor_caracteristicas['Avaliação'] = df['Avaliação']
    
    # Ponderando as características
    vetor_caracteristicas['Categoria_' + categoria_produto] *= 2  # Duplica o peso da categoria do produto
    
    vetor_caracteristicas = vetor_caracteristicas.values
    
    # Treinando o modelo KNN
    knn_model = NearestNeighbors(n_neighbors=3, algorithm='brute', metric='manhattan')
    knn_model.fit(vetor_caracteristicas)
    
    # Encontrando os vizinhos mais próximos
    produto_index = list(produtos_categoria).index(produto)
    _, indices = knn_model.kneighbors([vetor_caracteristicas[produto_index]])
    
    # Extraindo os produtos similares (excluindo o próprio produto)
    produtos_similares = [produtos_categoria[i] for i in indices.flatten()[1:]]
    
    return produtos_similares

# Criando a interface gráfica
def recomendar_produto():
    produto_comprado = entry_produto.get()
    produtos_recomendados = encontrar_produto_similar(produto_comprado)
    label_resultado.config(text=f"Produtos similares ao '{produto_comprado}': {', '.join(produtos_recomendados)}")

# Configurações da janela
root = tk.Tk()
root.title("Sistema de Recomendação de Produtos Eletrônicos")
root.geometry("400x200")

# Elementos da interface
label_produto = tk.Label(root, text="Produto Comprado:")
label_produto.pack(pady=5)

entry_produto = tk.Entry(root, width=30)
entry_produto.pack(pady=5)

button_recomendar = tk.Button(root, text="Recomendar", command=recomendar_produto)
button_recomendar.pack(pady=5)

label_resultado = tk.Label(root, text="")
label_resultado.pack(pady=5)

# Rodar a interface
root.mainloop()
